In [23]:
import librosa, librosa.display
import pandas as pd
import numpy as np
from numpy  import array
from itertools import tee, islice, chain
from math import atan2,degrees

In [24]:
# read audio file 
y, sr = librosa.load('D:/learning/DataScience/data/musicFiles/Bulleya - DJ LIJO & VIKAS J REMIX.mp3',duration = 60) 

In [25]:
# approach 1 - onset detection and dynamic programming
tempo, beat_times = librosa.beat.beat_track(y, sr=sr)
print(beat_times)
print(len(beat_times))

C:\ProgramData\Anaconda3\lib\site-packages\scipy\fftpack\basic.py:160: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  z[index] = x


[   3   32   59   87  114  142  169  197  224  251  279  307  334  362
  389  416  444  471  499  527  554  582  609  637  664  692  719  746
  774  801  829  857  888  918  946  973 1001 1031 1061 1090 1119 1147
 1174 1202 1230 1259 1289 1317 1345 1373 1401 1428 1456 1483 1511 1537
 1565 1592 1621 1650 1679 1709 1736 1764 1791 1819 1846 1873 1901 1929
 1956 1983 2011 2038 2066 2093 2121 2148 2176 2203 2231 2258 2286 2313
 2341 2368 2395 2423 2451 2478 2506 2533]
92


C:\ProgramData\Anaconda3\lib\site-packages\mkl_fft\_numpy_fft.py:1044: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  output = mkl_fft.rfftn_numpy(a, s, axes)


In [26]:
hop_length = 512
start_bpm=120.0
trim=True
tightness=100 
onset_envelope = librosa.onset.onset_strength(y=y,sr=sr,hop_length=hop_length,aggregate=np.median)
print(len(onset_envelope))
print(onset_envelope)

2584
[0.         0.         0.         ... 0.         0.         0.01287328]


In [27]:
test = []
for i in  beat_times:
    test.append(onset_envelope[i])


In [28]:
print(len(test))
print(test)

92
[25.624122297578648, 7.06219968326282, 0.49640956812781956, 6.073085729115821, 6.665821159354357, 6.859649373449226, 0.30925970768227806, 7.040692522770002, 10.564315905845795, 5.927920534688825, 0.0, 6.475012575107187, 8.595871050849972, 6.018549626162066, 7.308580460597903, 23.69122360312343, 11.738307609342364, 5.8123373695538305, 2.2777551254101036, 4.973458338925541, 6.054502306640206, 4.848484492250565, 7.442551537441592, 5.292863262926314, 7.288112554139683, 4.1086463943965015, 3.975465696183001, 5.942211859186049, 6.566096907654419, 5.907455346272831, 4.842035526699135, 2.3035150636929753, 0.0, 1.3038086679353755, 4.130711927432985, 2.4095522472972313, 2.306704472804671, 0.01230827244822752, 0.4687366869825009, 1.8218609174225264, 0.3467195807506682, 0.5156436454873714, 3.1480768969846067, 0.0, 0.9825200001921264, 0.0, 1.5007996845705895, 2.83862117293104, 2.3183496601881117, 1.346527895557069, 0.0, 1.4921037397084551, 0.004502945120163915, 0.31655718519719933, 3.83785248651

In [29]:
# Determines the angle of a straight line drawn between point one and two.
def getAngleBetweenTwoPoints(p1x,p1y,p2x,p2y):
    xDiff = p2x - p1x
    yDiff = p2y - p1y
    angle = degrees(atan2(yDiff, xDiff))
    if(angle >= 0):
        return angle
    else:
        return (angle+360)
def getPreviousAndNext(iterable_data):
    prevs, items, nexts = tee(iterable_data, 3)
    prevs = chain([None], prevs)
    nexts = chain(islice(nexts, 1, None), [None])
    return zip(prevs, items, nexts)
def getArrayOfAnglesBetweenConsecutivepointsInArray(iterable_data):
    xx = 1
    yy = 2
    np_arr =  np.array([])
    for previous, item, nxt in getPreviousAndNext(iterable_data):
        if(nxt == None):
            np_arr = np.append(np_arr,0)
        else:
            tempvar = getAngleBetweenTwoPoints(xx,item,yy,nxt)
            np_arr = np.append(np_arr,tempvar)
    return np_arr

In [30]:
print(getArrayOfAnglesBetweenConsecutivepointsInArray(test))

[273.08375623 278.65985967  79.83386441  30.65672456  10.96951582
 278.67990991  81.55012215  74.15612296 282.17137738 279.57525968
  81.22061067  64.75578585 291.20623936  52.21804229  86.50698742
 274.7823194  279.57835262 285.79720903  69.6471246   47.23019409
 309.66467878  68.9185959  294.94708226  63.38040608 287.45931105
 352.41394939  63.04875674  31.95942256 326.6294389  313.18582798
 291.50098078 293.46658797  52.51238177  70.51907349 300.15674272
 354.12790287 293.54971563  24.53330342  53.53447078 304.1333902
   9.58811968  69.19938348 287.6226714   44.49483271 315.50516729
  56.32402529  53.22247811 332.51332435 315.81872718 306.5994551
  56.17021668 303.90991601  17.33075378  74.14617427  49.3316006
 281.69816016  45.20475264  55.89769026   5.60907996 290.18027277
  78.21449423 309.65924956  24.49052323  63.53303422 322.94024437
 289.27245815  75.8261365  295.88941861 355.76583988  48.66827153
 296.43138304  65.3510389  302.099128   296.84780416  75.91672302
 304.22452749